### Streamlit app code with chat history

In [ ]:
# This is a streamlit app with chat history

from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock

import streamlit as st
import boto3

st.set_page_config(page_title="StreamlitChatMessageHistory", page_icon="📖")
st.title("📖 StreamlitChatMessageHistory")

"""
A basic example of using StreamlitChatMessageHistory to help LLMChain remember messages in a conversation.
The messages are stored in Session State across re-runs automatically. You can view the contents of Session State
in the expander below. View the
[source code for this app](https://github.com/langchain-ai/streamlit-agent/blob/main/streamlit_agent/basic_memory.py).
"""

# Set up memory
# msgs = StreamlitChatMessageHistory(key="langchain_messages")
msgs = StreamlitChatMessageHistory()

print("MSGS SET")
# memory = ConversationBufferMemory(chat_memory=msgs)
# memory = ConversationBufferMemory(return_messages=True)
memory = ConversationBufferMemory(memory_key="history", chat_memory=msgs)


if len(msgs.messages) == 0:
    msgs.add_ai_message("How can I help you?")

view_messages = st.expander("View the message contents in session state")

# initiate Bedrock
bedrock_run_time = boto3.client(service_name='bedrock-runtime')


inference_modifier = {
    "temperature": 1,
    "top_p": .999,
    "top_k": 250,
    "max_tokens_to_sample": 300,
    "stop_sequences": ["\n\nHuman:"]
}
bedrock_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock_run_time,
    model_kwargs=inference_modifier,
)


# Get an OpenAI API Key before continuing   
# if "openai_api_key" in st.secrets:
#     openai_api_key = st.secrets.openai_api_key
# else:
#     openai_api_key = st.sidebar.text_input("OpenAI API Key", type="password")
# if not openai_api_key:
#     st.info("Enter an OpenAI API Key to continue")
#     st.stop()

# Set up the LLMChain, passing in memory
template = """You are an AI chatbot having a conversation with a human.

{history}
Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)
# llm_chain = LLMChain(llm=OpenAI(openai_api_key=openai_api_key), prompt=prompt, memory=memory)

bedrock_chain = LLMChain(
    llm=bedrock_llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)


# Render current messages from StreamlitChatMessageHistory
for msg in msgs.messages:
    st.chat_message(msg.type).write(msg.content)

# If user inputs a new prompt, generate and draw a new response
if prompt := st.chat_input():
    st.chat_message("human").write(prompt)
    # Note: new messages are saved to history automatically by Langchain during run
    response = bedrock_chain.run(prompt)
    st.chat_message("ai").write(response)

# Draw the messages at the end, so newly generated ones show up immediately
with view_messages:
    """
    Memory initialized with:
    ```python
    msgs = StreamlitChatMessageHistory(key="langchain_messages")
    memory = ConversationBufferMemory(chat_memory=msgs)
    ```

    Contents of `st.session_state.langchain_messages`:
    """
    view_messages.json(st.session_state.langchain_messages)

### Athena SQL chat -without memory

In [2]:
import boto3
from botocore.config import Config
from langchain import PromptTemplate,SagemakerEndpoint,SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from sqlalchemy import create_engine

"""
Here we will build the required parameter to connect athena and query database.
1. Data is stored in S3 and metadata in Glue metastore.
2. Create a profille which will have access to the required service.
3. if the database exists and s3 buckets exists use them else create.

"""
region = 'us-east-1'
athena_url = f"athena.{region}.amazonaws.com" 
athena_port = '443' #Update, if port is different
athena_db = 'demo-emp-deb-2' #from user defined params
glue_databucket_name='athena-query-bucket-bharsrid'
s3stagingathena = f's3://{glue_databucket_name}/athenaresults/' 
athena_wkgrp = 'primary' 
athena_connection_string = f"awsathena+rest://@{athena_url}:{athena_port}/{athena_db}?s3_staging_dir={s3stagingathena}/&work_group={athena_wkgrp}"

"""
Under the hood, LangChain uses SQLAlchemy to connect to SQL databases. 
The SQLDatabaseChain can therefore be used with any SQL dialect 
supported by SQLAlchemy, such as MS SQL, MySQL, MariaDB, PostgreSQL, 
Oracle SQL, and SQLite. 
"""
print(athena_connection_string)
athena_engine = create_engine(athena_connection_string, echo=True)
athena_db_connection = SQLDatabase(athena_engine)


awsathena+rest://@athena.us-east-1.amazonaws.com:443/demo-emp-deb-2?s3_staging_dir=s3://athena-query-bucket-bharsrid/athenaresults//&work_group=primary


In [6]:
def get_schema(_):
    return athena_db_connection.get_table_info()

In [7]:
def run_query(query):
    return athena_db_connection.run(query)

In [8]:
inference_modifier = {
    "temperature": 1,
    "top_p": .999,
    "top_k": 250,
    "max_tokens_to_sample": 300,
    "stop_sequences": ["\n\nSQL Query:"]
}

In [11]:
from langchain.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

chat = BedrockChat(model_id="anthropic.claude-v2", model_kwargs=inference_modifier)

# model = ChatOpenAI()

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | chat.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [12]:
sql_response.invoke({"question": "How many employees are there?"})



2023-11-02 18:07:28,103 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:07:28,106 INFO sqlalchemy.engine.Engine [generated in 0.00235s] {'param_1': 3}
2023-11-02 18:07:29,578 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:07:29,579 INFO sqlalchemy.engine.Engine [generated in 0.00105s] {'param_1': 3}


/opt/conda/lib/python3.8/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:

CREATE EXTERNAL TABLE details (
	employee_id INT,
	`first name` STRING,
	`last name` STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'field.delim' = ','
)
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://employee-db-genai-demo/employee/Details/'
TBLPROPERTIES (
	'CrawlerSchemaDeserializerVersion' = '1.0',
	'CrawlerSchemaSerializerVersion' = '1.0',
	'UPDATED_BY_CRAWLER' = 'employee-db-demo-crawler',
	'areColumnsQuoted' = 'false',
	'averageRecordSize' = '18',
	'classification' = 'csv',
	'columnsOrdered' = 'true',
	'compressionType' = '

' SELECT COUNT(DISTINCT employee_id) \nFROM details'

In [13]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

In [18]:
full_chain = (
    RunnablePassthrough.assign(query=sql_response)
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: athena_db_connection.run(x["query"]),
    )
    | prompt_response
    | chat
)

In [19]:
full_chain.invoke({"question": "How many employees are there?"})

2023-11-02 18:16:03,233 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:16:03,236 INFO sqlalchemy.engine.Engine [cached since 515.1s ago] {'param_1': 3}
2023-11-02 18:16:04,519 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:16:04,520 INFO sqlalchemy.engine.Engine [cached since 514.9s ago] {'param_1': 3}


/opt/conda/lib/python3.8/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 

Human: Based on the table schema below, write a SQL query and just the SQL, nothing else, that would answer the user's question.:

CREATE EXTERNAL TABLE details (
	employee_id INT,
	`first name` STRING,
	`last name` STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
	'field.delim' = ','
)
STORED AS INPUTFORMAT 'org.apache.hadoop.mapred.TextInputFormat' OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://employee-db-genai-demo/employee/Details/'
TBLPROPERTIES (
	'CrawlerSchemaDeserializerVersion' = '1.0',
	'CrawlerSchemaSerializerVersion' = '1.0',
	'UPDATED_BY_CRAWLER' = 'employee-db-demo-crawler',
	'areColumnsQuoted' = 'false',
	'averageRecordSize' = '18',
	'classification' = 'csv',
	'columnsOrdered' = 'true',
	'compressionType' = '

2023-11-02 18:16:06,906 INFO sqlalchemy.engine.Engine SELECT details.employee_id, details."first name", details."last name" 
FROM details LIMIT %(param_1)s
2023-11-02 18:16:06,907 INFO sqlalchemy.engine.Engine [cached since 518.8s ago] {'param_1': 3}
2023-11-02 18:16:07,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-02 18:16:07,235 INFO sqlalchemy.engine.Engine  SELECT COUNT(*)
FROM details;
2023-11-02 18:16:07,236 INFO sqlalchemy.engine.Engine [generated in 0.00090s] {}
2023-11-02 18:16:08,353 INFO sqlalchemy.engine.Engine SELECT location.employee_id, location.location 
FROM location LIMIT %(param_1)s
2023-11-02 18:16:08,354 INFO sqlalchemy.engine.Engine [cached since 518.8s ago] {'param_1': 3}
2023-11-02 18:16:08,528 INFO sqlalchemy.engine.Engine COMMIT


AIMessage(content=' Based on the details table schema and sample data provided, there are 3 rows in the details table. The SQL query performs a COUNT(*) to return the total number of rows, which is 7 according to the SQL response. Therefore, there are 7 employees in the details table.')

### SQL Memory from documenation

In [ ]:
from langchain.memory.chat_message_histories import SQLChatMessageHistory

chat_message_history = SQLChatMessageHistory(
    session_id="test_session", connection_string="sqlite:///sqlite.db"
)

chat_message_history.add_user_message("Hello")
chat_message_history.add_ai_message("Hi")

chat_message_history.messages